## Prerequisites 

### Imports & Env

In [3]:
import os
import json 

import numpy as np
import pandas as pd

In [4]:
data_path = "data"
os.environ['data_path'] = data_path

### Download Data

In [ ]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1I03N5I_JSqYnCIX116TeqPVEpvfhCCOs", output=data_path)

Retrieving folder list


Processing file 1s0vmv5KIX5BPBiJqkJO_0ofTBgA6EfAz q1-test.json
Processing file 1oJbleoEJZ4V7KVTzlfyePTzfKi_T-gQJ q1-train.json
Processing file 1WG-fGK3fT0kVHknkB84DGvm9jLJXUmr7 q2-test.json
Processing file 1YCHEo3QjCZX4nVjXHOEi_u9LV_x4kmiN q2-train-hidden.json
Processing file 19dLi_xVDUal7Cr9z4V6arEXpVTtWCbcf q2-train.json
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1s0vmv5KIX5BPBiJqkJO_0ofTBgA6EfAz
To: /content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q1-test.json
100%|██████████| 108k/108k [00:00<00:00, 29.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oJbleoEJZ4V7KVTzlfyePTzfKi_T-gQJ
To: /content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q1-train.json
100%|██████████| 5.08M/5.08M [00:00<00:00, 261MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WG-fGK3fT0kVHknkB84DGvm9jLJXUmr7
To: /content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q2-test.json
100%|██████████| 108k/108k [00:00<00:00, 49.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YCHEo3QjCZX4nVjXHOEi_u9LV_x4kmiN
To: /content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q2-train-hidden.json
100%|██████████| 5.08M/5.08M [00:00<00:00, 98.1MB/s]
Downloading...
From: https://d

['/content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q1-test.json',
 '/content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q1-train.json',
 '/content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q2-test.json',
 '/content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q2-train-hidden.json',
 '/content/drive/MyDrive/Shared with Sabas/Stoch-Project-Fall-2022/data/q2-train.json']

## Data 

### Load

In [5]:
# Get a numpy array from json data
def get_json_list_data(path):
  with open(path) as json_file:
    json_data = json.load(json_file)
    return np.array(json_data)

In [6]:
train_data_path = os.path.join(data_path, "q1-train.json")
test_data_path = os.path.join(data_path, "q1-test.json")

train_data = get_json_list_data(train_data_path)
test_data = get_json_list_data(test_data_path)

In [7]:
train_data[0]

'dacfadagagbdfggbfafadabdfacfcaebbgggafbfgfbgagaggefebafbgdfbcggagdccfaadfdgagcadbbgegcdgcddgfefdddaacgcecedgegccagefggcfgdbgdacdcdcdgccgcaccafdfabbgcabgadcgafgecacfdfbdfafdegaadcdagafaddccfcddcfbebgdbbfadeaeadcdafcbagbfcfddfgcfcdagcedbbgfababcgadeeefcgdcdgcdacafcbcdfbfdceagabcdabccbcfgecccdfgecbagacdgafdcccaacgccdcagabccdfdcfgggaafafafgeegcaafcebdacagcdadcfdgcfdffbeagafdbefafeagbeagadaeadccaffcaacbafaeagafcgdeadcbbcggaacfafafbcccecgagcgadcfbfafgdabagaddcddfdabggacfggaagagggacddccfcfdfccfdebfcaac'

## Baum–Welch

### Initialization

In [8]:
observations = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g'])
observations_map = {v: i for i, v in enumerate(observations)}
print(observations_map)

states = np.array([0, 1, 2, 3, 4])
available_edges = np.array([
    (0,1), (0,2),
    (1,0), (1,1), (1,3),
    (2,0), (2,3), (2,4),
    (3,1), (3,2),
    (4,2), (4,4)
    ])

O = observations.shape[0]
S = states.shape[0]
print("Number of states:", S, "Number of observations:", O)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6}
Number of states: 5 Number of observations: 7


In [9]:
# Initialize transition matrix based on graph

A = np.zeros((S, S))
for u, v in available_edges:
  A[u][v] = 1
  
row_sum = A.sum(axis=1, keepdims=True)
A = A / row_sum

print("A:\n", A)

A:
 [[0.         0.5        0.5        0.         0.        ]
 [0.33333333 0.33333333 0.         0.33333333 0.        ]
 [0.33333333 0.         0.         0.33333333 0.33333333]
 [0.         0.5        0.5        0.         0.        ]
 [0.         0.         0.5        0.         0.5       ]]


In [10]:
# Initialize initial probabilities

pi = np.random.rand(S)
pi = pi / pi.sum()
print("pi:\n", pi)

pi:
 [0.08153233 0.32672606 0.01382806 0.3271354  0.25077814]


In [11]:
# Initialize observations probabilities

B = np.random.rand(S*O).reshape((S, O))

row_sum = B.sum(axis=1, keepdims=True)
B = B / row_sum

print("B:\n", B)

B:
 [[0.13072066 0.09902095 0.19817141 0.06517494 0.21678435 0.24864934
  0.04147834]
 [0.13574754 0.14971234 0.19523411 0.25952967 0.07697743 0.08906728
  0.09373163]
 [0.16381821 0.11238    0.23783329 0.17049454 0.04361586 0.18948122
  0.08237688]
 [0.07091065 0.1019264  0.15886519 0.1567162  0.09630264 0.25450249
  0.16077643]
 [0.09233505 0.22362096 0.143276   0.02062903 0.2221348  0.19240027
  0.10560389]]


### Utility functions

In [12]:
def get_observations_from_str(s):
  return [observations_map[y] for y in s]

### Algorithm

In [202]:
import copy

class BaumWelch:
  
  def __init__(self, A, B, pi):
    self.A = copy.deepcopy(A)
    self.B = copy.deepcopy(B)
    self.pi = copy.deepcopy(pi)

    self.S = A.shape[0]
    self.O = B.shape[1]

    self.alpha = None
    self.beta = None

  def forward(self, Y):
    """
    forward method will calculate alpha values which are defined as P(Y1=y1, ..., Yt=yt, Xt = i | params)
    Y should be a numpy array in this format: (n_samples, length_of_each_sample)
    """
    samples_count = Y.shape[0]
    T = Y.shape[1]

    self.alpha = np.zeros((samples_count, T, self.S))
    self.g = np.zeros((samples_count, T))

    for idx in range(samples_count):
      for t in range(T):
        for i in range(self.S):
          if t == 0:  
            self.alpha[idx][t][i] = self.B[i][Y[idx, t]] * self.pi[i]
          else:
            self.alpha[idx][t][i] = self.B[i][Y[idx,t]] * self.alpha[idx][t-1].dot(self.A[:, i])
          
          # self.g[t] += self.alpha[t][i]


        self.g[idx][t] = self.alpha[idx, t, :].sum()
        self.alpha[idx][t] /= self.g[idx][t]
    

  def backward(self, Y):
    """
    backward method will calculate beta values which are defined as P(Yt=yt, ..., YT=yT | Xt = i , params)
    Y should be a numpy array in this format: (n_samples, length_of_each_sample)
    """
    samples_count = Y.shape[0]
    T = Y.shape[1]

    self.beta = np.zeros((samples_count, T, self.S))

    for idx in range(samples_count):
      for t in range(T-1, -1, -1):
        for i in range(self.S):
          if t == T-1 :
            self.beta[idx][t][i] = 1
          else:
            # self.beta[t][i] = sum([self.A[i][j] * self.beta[t+1][j] * self.B[j][Y[t+1]] for j in range(S)])
            self.beta[idx][t][i] = (self.A[i] * self.beta[idx][t+1]).dot(self.B[:,Y[idx][t+1]])

        self.beta[idx][t] /= self.g[idx][t]

  def update(self, Y):
    """ 
    update method will first use alpha and beta values to calculate lmbda and epsillon.
    lmbda is defined as P(Xt = i | Y, params) and epsillon is defined as P(Xt=i, X{t+1}=j | Y, params).
    Then it will calculate pi, A, and B using epsillon and lmbda.
    Y should be a numpy array in this format: (n_samples, length_of_each_sample)
    """
    samples_count = Y.shape[0]
    T = Y.shape[1]

    lmbda = np.zeros((samples_count, T, self.S))
    epsillon = np.zeros((samples_count, T-1, self.S, self.S))

    for idx in range(samples_count):
      for t in range(T):
        for i in range(S):
          lmbda[idx][t][i] = self.alpha[idx][t][i] * self.beta[idx][t][i]
        t_sum = lmbda[idx][t].sum()
        # print(lmbda[t], t_sum)
        if t_sum != 0:
          lmbda[idx][t] = lmbda[idx][t] / t_sum

    for idx in range(samples_count):
      for t in range(T-1):
        for i in range(self.S):
          for j in range(self.S):
            epsillon[idx][t][i][j] = self.alpha[idx][t][i] * self.A[i][j] * self.beta[idx][t+1][j] * self.B[j][Y[idx][t+1]]
        t_sum = epsillon[idx][t].sum()
        if t_sum != 0:
          epsillon[idx][t] = epsillon[idx][t] / t_sum

    self.pi = lmbda[:, 0].mean(axis=0)

    for i in range(self.S):
      for j in range(self.S):
        # self.A[i][j] = sum([epsillon[t][i][j] for t in range(T-1)]) /  sum([lmbda[t][i] for t in range(T-1)])
        self.A[i][j] = epsillon[:, :-1, i, j].sum() /  lmbda[:, :-1,i].sum()

    for i in range(self.S):
      for v in range(self.O):
        # self.B[i][v] = sum([lmbda[t][i] for t in range(0, T) if Y[t] == v]) / sum([lmbda[t][i] for t in range(T)])
        self.B[i][v] = sum([sum([lmbda[idx, t, i] for t in range(0, T) if Y[idx][t] == v]) for idx in range(samples_count)]) / lmbda[:, :,i].sum()


  def iteration(self, Y):
    """ 
    Perform one iteration on algorithm
    Y should be a numpy array in this format: (n_samples, length_of_each_sample)
    """
    self.forward(Y)
    self.backward(Y)
    self.update(Y)

  def print(self, verbose=1):
    """ 
    Print parameters
    """
    print("pi:\n", self.pi)
    print("A:\n", self.A)
    print("B:\n", self.B)
    if verbose == 2:
      if self.alpha is not None:
        print("alpha:\n", self.alpha[:5])
      if self.beta is not None:
        print("beta:\n", self.beta[:5])


### Learn parameters

In [203]:
# Get all observations and convert them to numerical values
all_X = np.array([get_observations_from_str(train_data[i]) for i in range(len(train_data))])
print(all_X.shape)

(10000, 500)


In [204]:
bw = BaumWelch(A, B, pi)

In [208]:
# Train baum-welch for the first `samples_count` sequence of observations

iteration_count = 4
samples_count = 500
for _ in range(iteration_count):
    samples = all_X[:samples_count]
    print(samples.shape)
    bw.iteration(samples)
bw.print()


(500, 500)
(500, 500)
(500, 500)
(500, 500)
pi:
 [0.04310594 0.45705257 0.02352158 0.25159335 0.22472656]
A:
 [[0.         0.49951341 0.4984757  0.         0.        ]
 [0.31019073 0.30704072 0.         0.38082714 0.        ]
 [0.32433766 0.         0.         0.41711731 0.25647345]
 [0.         0.4853554  0.5126746  0.         0.        ]
 [0.         0.         0.61320638 0.         0.38473914]]
B:
 [[0.11529812 0.09064547 0.14641015 0.08255369 0.14160343 0.32302847
  0.10046068]
 [0.24730434 0.10830922 0.23855036 0.27727345 0.03033892 0.02520146
  0.07302224]
 [0.45986091 0.07467728 0.23587491 0.11980931 0.01884799 0.03318667
  0.05774294]
 [0.04969152 0.07429088 0.0932284  0.15809775 0.05023679 0.26324186
  0.3112128 ]
 [0.14189708 0.17306177 0.11143907 0.02348532 0.13554697 0.16525591
  0.24931388]]


In [210]:
np.savetxt("A_500_4.csv", bw.A, delimiter=",") # Transitions
np.savetxt("B_500_4.csv", bw.B, delimiter=",") # Emissions
np.savetxt("pi_500_4.csv", bw.pi, delimiter=",") # Initial probs

### Inference

In [246]:
# Use forward method to calculate the probability of each sequence

p_list = []
for y in test_data:
    Y = np.array(get_observations_from_str(y)).reshape(1, -1)
    bw.forward(Y)
    T = len(Y)
    p = bw.alpha[0][T-1].sum() * bw.g[0][T-1]
    print(p)
    p_list.append([p])

np.savetxt("seq_prob.csv", np.array(p_list), delimiter=",")

0.1295907331708731
0.1733901828930432
0.1733901828930432
0.1693883178579418
0.1693883178579418
0.1127495627253183
0.1693883178579418
0.1295907331708731
0.17320787432653373
0.1733901828930432
0.1295907331708731
0.1295907331708731
0.1295907331708731
0.1127495627253183
0.1295907331708731
0.17320787432653373
0.1733901828930432
0.1733901828930432
0.1693883178579418
0.17320787432653373
0.1693883178579418
0.17815931627915213
0.1127495627253183
0.1693883178579418
0.17320787432653373
0.1295907331708731
0.17815931627915213
0.1733901828930432
0.1733901828930432
0.17320787432653373
0.1733901828930432
0.1693883178579418
0.17320787432653373
0.1295907331708731
0.06351401274713826
0.1127495627253183
0.17815931627915213
0.1733901828930432
0.1733901828930432
0.1733901828930432
0.17815931627915213
0.17320787432653373
0.17815931627915213
0.17320787432653373
0.1733901828930432
0.17815931627915213
0.1693883178579418
0.1295907331708731
0.17815931627915213
0.1295907331708731
0.1295907331708731
0.1693883178579

## Document

The "Baum-Welch" algorithm as stated in [1] is used to estimate model parameters. 
Check BaumWelch class methods' doc strings to see what each method does.

### Results

**Part1-a**

The trained parameters can be found in `A_500_4.csv`, `B_500_4.csv`, `pi_500_4.csv`. Since the code was a little slow, only 500 evidence sequences were used with 4 iterations.

**Part1-b**

The probabilities are in `seq_prob.csv`.

### Challenges

1. The `alpha`, `beta`, and estimated parameters vanished to zero when number of states and iterations increased. I used "scaling" solution mentioned in [2] to solve this problem.

2. It is slow! I tried to make it a little faster by some vectorization. The thing is that numpy `sum` for doubles has lower accuracy compared to python `sum`. So the results are a little different from when I used python `sum`.

### Resources

1. https://en.wikipedia.org/wiki/Baum–Welch_algorithm
2. https://courses.grainger.illinois.edu/ECE417/fa2021/lectures/lec16.pdf